# Ch.12 データ加工に挑戦する

In [94]:
import math
import pandas as pd
from IPython.display import display
import hashlib
import chardet
import json
import sqlite3

## Knock111: "よくある"Excelデータに挑戦

「セル結合」されたデータ

- 入力データ
  - 12-1.xlsx
- 加工要件
  - 市町村区単位で1レコード化
  - 欠損値が存在いない状態
  - cvsファイルに出力
- 出力データ
  - 12-1_out.cvs
- ヒント
  - 入力データ・要件・出力イメージを確認して必要な処理をする
  - PandasでExcelファイルを読み込む
  - 男性・女性を横に結合して、不要なカラムを除去する
  - 欠損値の補完
  - データの確認、出力

In [2]:
# Excelファイルの読み込み
input_data = pd.read_excel("../support/12章/12-1.xlsx")
print(input_data.shape)
display(input_data)

(8, 3)


,都道府県,市区町村,人数（男性、女性）
0,東京都,新宿区,12
1,NaN,NaN,14
2,NaN,豊島区,15
3,NaN,NaN,13
4,神奈川県,横浜市,8
5,NaN,NaN,9
6,NaN,横須賀市,5
7,NaN,NaN,2


In [3]:
# 「人数」項目を男女に分ける
men = input_data['人数（男性、女性）'][0::2]
women = input_data['人数（男性、女性）'][1::2]

# indexのリセット（元のオブジェクトの変更、元のindexの削除）
men.reset_index(inplace=True, drop=True)
women.reset_index(inplace=True, drop=True)

display(men)
display(women)

0    12
1    15
2     8
3     5
Name: 人数（男性、女性）, dtype: int64

0    14
1    13
2     9
3     2
Name: 人数（男性、女性）, dtype: int64

In [4]:
# 元データのコピー
output_data = input_data[::2].copy()

# indexのリセット（元のオブジェクトの変更、元のindexの削除）
output_data.reset_index(inplace=True, drop=True)

display(output_data)

,都道府県,市区町村,人数（男性、女性）
0,東京都,新宿区,12
1,NaN,豊島区,15
2,神奈川県,横浜市,8
3,NaN,横須賀市,5


In [5]:
# 「男性」、「女性」項目の追加
output_data["男性"] = men
output_data["女性"] = women

# 不要になった「人数（男性、女性）」項目を削除
output_data.drop("人数（男性、女性）", axis=1, inplace=True)

display(output_data)

,都道府県,市区町村,男性,女性
0,東京都,新宿区,12,14
1,NaN,豊島区,15,13
2,神奈川県,横浜市,8,9
3,NaN,横須賀市,5,2


In [6]:
# 欠損している都道府県を埋める
output_data.iat[1, 0] = output_data.iat[0, 0]  # 東京都
output_data.iat[3, 0] = output_data.iat[2, 0]  # 神奈川県

display(output_data)

,都道府県,市区町村,男性,女性
0,東京都,新宿区,12,14
1,東京都,豊島区,15,13
2,神奈川県,横浜市,8,9
3,神奈川県,横須賀市,5,2


In [7]:
# csvファイルに出力
output_data.to_csv("12-1_out.csv", index=False)

## Knock112: Excelの社員マスタ加工に挑戦

Excelの社員マスタの加工

- 入力データ
  - 12-2.xlsx
- 加工条件
  - 重複する社員は更新日が新しいデータを優先する
  - 重複判定は、氏名・生年月日でに行う
  - 登録日は昇順で整形して、csvファイルに出力する
- 出力データ
  - 12-2_out.csv
- ヒント
  - 入力データ・要件・出力イメージを確認して、必要な処理を考える
  - PandasでExcelファイルを読み込む
  - 重複するデータを特定する
  - 優先するデータを選定する
  - データの確認、出力

In [9]:
# Excelファイルの読み込む
input_data = pd.read_excel("../support/12章/12-2.xlsx")
display(input_data)

,社員名,生年月日,部署,役職,更新日
0,田中 正,1975-10-09,A部,課長,2021-10-12
1,水野 メイサ,1981-02-23,C部,課長,2020-05-08
2,齊藤 隆,2001-01-23,B部,NaN,2021-05-08
3,茂木 新人,2002-08-23,A部,NaN,2021-04-01
4,篠山 雅功,1992-07-02,A部,課長,2022-03-02
5,水野 メイサ,1981-02-23,C部,部長,2021-12-08
6,白鳥 りえ,1999-04-11,B部,NaN,2022-02-01
7,田中 正,1975-10-09,A部,部長,2022-03-02
8,篠山 雅功,1992-07-02,B部,NaN,2021-01-15


In [10]:
# 役職の「NaN」を「-」に置換する
output_data = input_data.copy()
output_data["役職"].fillna("-", inplace=True)
display(output_data)

,社員名,生年月日,部署,役職,更新日
0,田中 正,1975-10-09,A部,課長,2021-10-12
1,水野 メイサ,1981-02-23,C部,課長,2020-05-08
2,齊藤 隆,2001-01-23,B部,-,2021-05-08
3,茂木 新人,2002-08-23,A部,-,2021-04-01
4,篠山 雅功,1992-07-02,A部,課長,2022-03-02
5,水野 メイサ,1981-02-23,C部,部長,2021-12-08
6,白鳥 りえ,1999-04-11,B部,-,2022-02-01
7,田中 正,1975-10-09,A部,部長,2022-03-02
8,篠山 雅功,1992-07-02,B部,-,2021-01-15


In [11]:
# データを更新日で並び替える
output_data.sort_values("更新日", ascending=True, inplace=True)
display(output_data)

,社員名,生年月日,部署,役職,更新日
1,水野 メイサ,1981-02-23,C部,課長,2020-05-08
8,篠山 雅功,1992-07-02,B部,-,2021-01-15
3,茂木 新人,2002-08-23,A部,-,2021-04-01
2,齊藤 隆,2001-01-23,B部,-,2021-05-08
0,田中 正,1975-10-09,A部,課長,2021-10-12
5,水野 メイサ,1981-02-23,C部,部長,2021-12-08
6,白鳥 りえ,1999-04-11,B部,-,2022-02-01
4,篠山 雅功,1992-07-02,A部,課長,2022-03-02
7,田中 正,1975-10-09,A部,部長,2022-03-02


In [13]:
# 重複するデータを特定して、新しいデータを保持する
output_data.drop_duplicates(
    subset=["社員名", "生年月日"],
    keep="last",
    inplace=True
)
display(output_data)

,社員名,生年月日,部署,役職,更新日
3,茂木 新人,2002-08-23,A部,-,2021-04-01
2,齊藤 隆,2001-01-23,B部,-,2021-05-08
5,水野 メイサ,1981-02-23,C部,部長,2021-12-08
6,白鳥 りえ,1999-04-11,B部,-,2022-02-01
4,篠山 雅功,1992-07-02,A部,課長,2022-03-02
7,田中 正,1975-10-09,A部,部長,2022-03-02


In [14]:
# csvファイルに出力
output_data.to_csv("12-2_out.csv", index=False)

## Knock113: 正規化に挑戦

Excelの仕入れデータの正規化

- 入力データ
  - 12-3.csv
- 加工要件
  - 仕入先データ、商品データ、仕入れデータを分離して、第2正規形に加工する
  - 仕入先のキーはF1~とする
  - 商品のキーはP1~とする
  - それぞれのデータをcsvファイルに出力する
- 出力データ
  - 12-3order.csv, 12-3_farmer.csv, 12-3_product.csv
- ヒント
  - 入力データ、要件、出力イメージを確認して、必要な処理を考える
  - PandasでExcelファイルを読み込む
  - 仕入先を抽出して、キーに置き換える
  - 商品を抽出して、キーに置き換える
  - データの確認、出力

In [15]:
# Excelファイルを読み込む
input_data = pd.read_excel("../support/12章/12-3.xlsx")
display(input_data)

,仕入先,仕入先TEL,商品,販売単価,入荷日
0,A農家,03-4444-4444,きゃべつ,100,2019-07-01
1,NaN,NaN,れたす,80,2019-07-03
2,NaN,NaN,きゃべつ,100,2019-08-01
3,B農家,042-222-3333,もやし,20,2019-07-08


In [16]:
# セル結合の欠損値を埋める
# 今回はA農家のみ欠損なのでfillnaを用いることが可能
input_data["仕入先"].fillna(input_data["仕入先"][0], inplace=True)
input_data["仕入先TEL"].fillna(input_data["仕入先TEL"][0], inplace=True)
display(input_data)

,仕入先,仕入先TEL,商品,販売単価,入荷日
0,A農家,03-4444-4444,きゃべつ,100,2019-07-01
1,A農家,03-4444-4444,れたす,80,2019-07-03
2,A農家,03-4444-4444,きゃべつ,100,2019-08-01
3,B農家,042-222-3333,もやし,20,2019-07-08


In [17]:
# 仕入先を別のデータフレームに格納する
farmer_data = input_data[["仕入先", "仕入先TEL"]].copy()

# 重複するデータの整理する
farmer_data.drop_duplicates(inplace=True)

display(farmer_data)

,仕入先,仕入先TEL
0,A農家,03-4444-4444
3,B農家,042-222-3333


In [23]:
# 仕入先のキーを生成する
farmer_index = []
num = 0
for farmer in  farmer_data["仕入先"]:
    num += 1
    farmer_index.append(f"F{num}")

# 仕入先キー列を追加する
farmer_data.insert(0, "仕入先ID", farmer_index)

display(farmer_data)

,仕入先ID,仕入先,仕入先TEL
0,F1,A農家,03-4444-4444
3,F2,B農家,042-222-3333


In [24]:
# 商品データを別のデータフレームに格納する
product_data = input_data[["商品", "販売単価"]].copy()

# 重複データを整理する
product_data.drop_duplicates(inplace=True)

display(product_data)

,商品,販売単価
0,きゃべつ,100
1,れたす,80
3,もやし,20


In [25]:
# 商品のキーを生成する
product_index = []
num = 0
for product in product_data["商品"]:
    num += 1
    product_index.append(f"P{num}")

# 商品キー列を追加する
product_data.insert(0, "商品ID", product_index)

display(product_data)

,商品ID,商品,販売単価
0,P1,きゃべつ,100
1,P2,れたす,80
3,P3,もやし,20


In [30]:
# 取引テーブルをキーに変換する
order_data = pd.merge(
    input_data, farmer_data[["仕入先ID", "仕入先"]],
    on="仕入先",
    how="left"
)
order_data = pd.merge(
    order_data, product_data[["商品ID", "商品"]],
    on="商品",
    how="left"
)

# カラムの削除と順序入れ替え
order_data = order_data[["仕入先ID", "商品ID", "入荷日"]]

display(order_data)

,仕入先ID,商品ID,入荷日
0,F1,P1,2019-07-01
1,F1,P2,2019-07-03
2,F1,P1,2019-08-01
3,F2,P3,2019-07-08


In [31]:
# 各データフレームをcsvファイルに出力
order_data.to_csv("12-3_order.csv", index=False)
farmer_data.to_csv("12-3_farmer.csv", index=False)
product_data.to_csv("12-3_product.csv", index=False)

## Knock114: 外れ値の加工に挑戦

購買情報データに含まれる外れ値の加工

- 入力データ
  - 12-4.csv
- 加工要件
  - 外れ値の基準は「第3四分位数」を超えたもの
  - 外れ値の対処は「第3四分位数」を丸めた値
  - 金額の代表値データをcsvファイルに出力する
- 出力データ
  - 12-4_out.csv

In [32]:
# csvファイルの読み込み
input_data = pd.read_csv("../support/12章/12-4.csv")
display(input_data)

,ID,金額
0,0,2422
1,1,1089
2,2,6413
3,3,1210
4,4,3643
...,...,...
195,195,6171
196,196,2299
197,197,1331
198,198,2121


In [33]:
# 「金額」項目の統計量
input_data["金額"].describe().astype("int")

count        200
mean        9299
std        76251
min          363
25%         1331
50%         2541
75%         4840
max      1076449
Name: 金額, dtype: int64

In [34]:
# 第3四分位数を取得する
threshold = input_data["金額"].quantile(0.75)

# 第3ん四分位数を超えたデータを第3四分位数で置き換える
output_data = input_data.copy()
output_data.loc[input_data["金額"] > threshold, "金額"] = threshold

output_data["金額"].describe()

count     200.00000
mean     2896.34500
std      1486.26091
min       363.00000
25%      1331.00000
50%      2541.00000
75%      4840.00000
max      4840.00000
Name: 金額, dtype: float64

In [35]:
# csvファイルに出力する
output_data["金額"].describe().to_csv("12-4_out.csv")

## Knock115: 欠損値の補完に挑戦

顧客情報の欠損値の補完

- 入力データ
  - 12-5.csv
- 加工要件
  - 都道府県の欠損値は、同じ市区町村から補完する
  - 年齢の欠損値は、同じ市区町村の平均（切り捨て）で補完する
  - cvsファイルに出力する
- 出力データ
  - 12-5_out.csv
- ヒント
  - 入力データ、要件、出力データを確認して必要な処理を考える
  - PandasでExcelファイルを読み込む
  - 欠損項目を確認する
  - 市区町村単位で集計して、都道府県と年齢の平均を算出して欠損値を補完する
  - データの確認、出力

In [36]:
# Excelファイルの読み込み
input_data = pd.read_excel("../support/12章/12-5.xlsx")
display(input_data)

,顧客名,都道府県,市区町村,年齢
0,須賀ひとみ,東京,H市,20.0
1,岡田 敏也,神奈川,E市,23.0
2,芳賀 希,東京,A市,44.0
3,荻野 愛,神奈川,F市,21.0
4,栗田 憲一,神奈川,E市,49.0
5,梅沢 麻緒,東京,A市,18.0
6,相原 ひとり,東京,H市,NaN
7,新村 丈史,埼玉,B市,29.0
8,石川 まさみ,NaN,G市,33.0
9,小栗 正義,埼玉,G市,87.0


In [39]:
output_data = input_data.copy()
output_data.isnull().sum()

顧客名     0
都道府県    4
市区町村    0
年齢      5
dtype: int64

In [61]:
# 都道府県の欠損データを補完する
# 同じ市区町村から補完する
target_div = output_data.loc[output_data["都道府県"].isnull(), "市区町村"]
for division in target_div:
    output_data.loc[
        (output_data["都道府県"].isnull()) &
        (output_data["市区町村"] == division),
        "都道府県"
    ] = output_data.loc[
        ~(output_data["都道府県"].isnull()) &
        (output_data["市区町村"] == division),
        "都道府県"
    ].unique()[0]

display(output_data)

,顧客名,都道府県,市区町村,年齢
0,須賀ひとみ,東京,H市,20.0
1,岡田 敏也,神奈川,E市,23.0
2,芳賀 希,東京,A市,44.0
3,荻野 愛,神奈川,F市,21.0
4,栗田 憲一,神奈川,E市,49.0
5,梅沢 麻緒,東京,A市,18.0
6,相原 ひとり,東京,H市,NaN
7,新村 丈史,埼玉,B市,29.0
8,石川 まさみ,埼玉,G市,33.0
9,小栗 正義,埼玉,G市,87.0


In [62]:
# 年齢の欠損データを補完する
# 同じ市区町村の平均（切り捨て）で補完する
target_div = output_data.loc[output_data["年齢"].isnull(), "市区町村"]
for division in target_div:
    output_data.loc[
        (output_data["年齢"].isnull()) &
        (output_data["市区町村"] == division),
        "年齢"
    ] = math.floor(
        output_data.loc[output_data["市区町村"] == division, "年齢"].mean()
    )

display(output_data)

,顧客名,都道府県,市区町村,年齢
0,須賀ひとみ,東京,H市,20.0
1,岡田 敏也,神奈川,E市,23.0
2,芳賀 希,東京,A市,44.0
3,荻野 愛,神奈川,F市,21.0
4,栗田 憲一,神奈川,E市,49.0
5,梅沢 麻緒,東京,A市,18.0
6,相原 ひとり,東京,H市,52.0
7,新村 丈史,埼玉,B市,29.0
8,石川 まさみ,埼玉,G市,33.0
9,小栗 正義,埼玉,G市,87.0


In [63]:
# csvファイルに出力する
output_data.to_csv("12-5_out.csv", index=False)

## Knock116: データのスクランブル化に挑戦

個人情報などを取り扱う際に行う、データのスクランブル化

- スクランブル化
  - 個人情報の不要なデータについて個人を特定できない状態にすること
  - 漏洩リスクなどに対処する方法
  - ハッシュ化

---

- 入力データ
  - 12-6.xlsx
- 加工要件
  - 氏名をハッシュ値に置換する
  - ハッシュ化された氏名で集計を行い、購入金額の合計を算出する
  - 集計結果をcsvファイルに出力する
- 出力データ
  - 12-6_out.csv
- ヒント
  - 入力データ、要件、出力イメージを確認して必要な処理を考える
  - PandasでExcelファイルを読み込む
  - スクランブル前に氏名で購入金額の集計を実施する（検証用）
  - hashlibライブラリをインポートする
  - 氏名をハッシュアルゴリズム（sha256など）で置換する
  - ハッシュ化された氏名で購入金額の集計を実施する
  - データの確認、出力

In [64]:
# Excelデータの読み込み
input_data = pd.read_excel("../support/12章/12-6.xlsx")
display(input_data)

,氏名,購入金額
0,須賀ひとみ,2131
1,須賀ひとみ,5213
2,稲田 将也,3292
3,西脇 礼子,1122
4,栗田 憲一,4823
5,荻野 愛,924
6,相原 ひとり,1022
7,稲田 将也,3023
8,荻野 愛,8623
9,西脇 礼子,5712


In [65]:
# 検証用として、各人の合計購入金額を集計する
input_data.groupby("氏名").sum()

,購入金額
氏名,
内村 まさみ,5712
小口 豊,9452
岩佐 孝太郎,5818
栗田 憲一,4823
相原 ひとり,1022
稲田 将也,29026
荻野 愛,24506
西脇 礼子,23149
須賀ひとみ,8896


In [68]:
output_data = input_data.copy()

# 氏名をハッシュ化する
output_data["氏名"] = output_data["氏名"].apply(
    lambda x: hashlib.sha256(x.encode()).hexdigest()
)
# ハッシュ化した氏名で集計する
output_data.groupby("氏名").sum()

,購入金額
氏名,
061027bd9eb2c1109262ceb6b7067cd71a1811ecc2844d828acacf5555a738f9,5818
0d60d878da37366d7d67f103569520b03a6fcde49a2963a1997bea5b6b9b7b40,4823
0f8d4726fc84296eef6d8a9dbf631cb978caecd759a1582215200f98d8b82e9a,8896
15416e267a8fb81d36a34c02f84d3efefeaac0f5d085a7446555bba32f42e6ba,29026
1a6c9503a1f518faec9891fed43e0228b1d76404486a1d904b73b6b8ef8ea032,23149
51659db65801946f4a3e3f234eb74f240dd2181ee0da7fbe4a44c540835d206c,5723
5a33912e5b3a3fa4f7b53018bfbc1ef798a552bf06b7c1aad023029153d194f6,24506
7738026cec1b844dcfa4b4f3f1fc7d5f701a1d0d8a2a324393c237242035c395,5712
a92e8099c51126ed4fca5aaf2af7c75f2b0cafddc4c5f0212003f90730a861aa,9452


In [69]:
# csvファイルに出力する
output_data.groupby("氏名").sum().to_csv("12-6_out.csv")

## Knock117: 文字コードの自動判定に挑戦

提供されたcsvが「UTF_8」以外の「文字コード」で保存されていた場合の対処

- 入力データ
  - 12-7-1.csv
  - 12-7-2.csv
  - 12-7-3.csv
- 加工要件
  - 各ファイルを読み込み、文字化けなどがないことを確認する
  - すべてのデータを統合する
  - csvファイル（UTF-8）に出力する
- ヒント
  - 入力データ、要件、出力イメージを確認して必要な処理を考える
  - chardetライブラリでcsvファイルの文字コードを調べる
  - データフレームを結合する
  - データの確認、出力

In [75]:
# ファイルの文字コードを調べる

files = [
    "../support/12章/12-7-1.csv",
    "../support/12章/12-7-2.csv",
    "../support/12章/12-7-3.csv"
]
df = pd.DataFrame()

for file in files:
    # バイナリでファイルを開く
    with open(file, mode="rb") as f:
        contents = f.read()
        enc = chardet.detect(contents)["encoding"]
        # 判明した文字コードでファイルを読み込む
        df = pd.concat([df, pd.read_csv(file, encoding=enc)])

display(df)

,file,text
0,12-7-1,このファイルはSJISで記載されています。
0,12-7-2,このファイルはUTF-16で記載されています。
0,12-7-3,このファイルはEUC-JPで記載されています。


In [76]:
# csvファイルに出力する
df.to_csv("12-7_out.csv", index=False)

## Knock118: センサーデータの加工に挑戦

センサーデータの取り扱い

- 2つの異なるIoTセンサーのログを取得する
- 取得周期にズレが生じている
  - その時間ズレを加工する
  - 同じ時間軸になるようにセンサー値の補間

---

- 入力データ
  - 12-8-1.csv
  - 12-8-2.csv
- 加工要件
  - センサーログファイル2つを読み込む
  - 欠損箇所について線形補間を用いて補う
  - csvファイルに出力する
- ヒント
  - 入力データ、要件、出力イメージを確認して必要な処理を考える
  - Pandasでcsvファイルを読み込む
  - 相手の時間軸が欠損値となるので、線形補間を行う
    - dataframe.interpolate(method="linear")
  - データの確認、出力

In [79]:
# データの読み込む
sensor1 = pd.read_csv("../support/12章/12-8-1.csv", index_col=0)
sensor2 = pd.read_csv("../support/12章/12-8-2.csv", index_col=0)
display(sensor1)
display(sensor2)

,sensor_1,sensor_2
time_stamp,,
2020/12/17 19:41:05.411,-1485,0
2020/12/17 19:41:05.596,-1817,0
2020/12/17 19:41:05.795,-1863,0
2020/12/17 19:41:05.996,-1871,0
2020/12/17 19:41:06.199,-1931,0
...,...,...
2020/12/17 19:41:31.595,-2586,0
2020/12/17 19:41:31.796,-2522,0
2020/12/17 19:41:31.998,-1960,0


,sensor_3,sensor_4
time_stamp,,
2020/12/17 19:41:05.519,0,-1201
2020/12/17 19:41:05.703,0,-1536
2020/12/17 19:41:05.905,0,-1624
2020/12/17 19:41:05.098,0,-1638
2020/12/17 19:41:06.307,0,-1641
...,...,...
2020/12/17 19:41:31.695,-1624,0
2020/12/17 19:41:31.906,-1750,0
2020/12/17 19:41:31.098,-1843,0


In [80]:
# 2つのセンサー値を結合する
df_main = pd.concat([sensor1, sensor2], ignore_index=False)
display(df_main)

,sensor_1,sensor_2,sensor_3,sensor_4
time_stamp,,,,
2020/12/17 19:41:05.411,-1485.0,0.0,NaN,NaN
2020/12/17 19:41:05.596,-1817.0,0.0,NaN,NaN
2020/12/17 19:41:05.795,-1863.0,0.0,NaN,NaN
2020/12/17 19:41:05.996,-1871.0,0.0,NaN,NaN
2020/12/17 19:41:06.199,-1931.0,0.0,NaN,NaN
...,...,...,...,...
2020/12/17 19:41:31.695,NaN,NaN,-1624.0,0.0
2020/12/17 19:41:31.906,NaN,NaN,-1750.0,0.0
2020/12/17 19:41:31.098,NaN,NaN,-1843.0,0.0


In [84]:
# タイムスタンプで並び替える
df_main.sort_values("time_stamp", inplace=True)
display(df_main)

,sensor_1,sensor_2,sensor_3,sensor_4
time_stamp,,,,
2020/12/17 19:41:05.098,NaN,NaN,0.0,-1638.0
2020/12/17 19:41:05.411,-1485.0,0.0,NaN,NaN
2020/12/17 19:41:05.519,NaN,NaN,0.0,-1201.0
2020/12/17 19:41:05.596,-1817.0,0.0,NaN,NaN
2020/12/17 19:41:05.703,NaN,NaN,0.0,-1536.0
...,...,...,...,...
2020/12/17 19:41:31.998,-1960.0,0.0,NaN,NaN
2020/12/17 19:41:32.199,-1373.0,0.0,NaN,NaN
2020/12/17 19:41:32.307,NaN,NaN,-1941.0,0.0


In [85]:
# 欠損値の確認
df_main.isnull().sum()

sensor_1    136
sensor_2    136
sensor_3    136
sensor_4    136
dtype: int64

In [86]:
# 欠損値を線形補間する
df_main.interpolate(method="linear", inplace=True)
display(df_main)

,sensor_1,sensor_2,sensor_3,sensor_4
time_stamp,,,,
2020/12/17 19:41:05.098,NaN,NaN,0.000000,-1638.0
2020/12/17 19:41:05.411,-1485.0,0.0,0.000000,-1419.5
2020/12/17 19:41:05.519,-1651.0,0.0,0.000000,-1201.0
2020/12/17 19:41:05.596,-1817.0,0.0,0.000000,-1368.5
2020/12/17 19:41:05.703,-1840.0,0.0,0.000000,-1536.0
...,...,...,...,...
2020/12/17 19:41:31.998,-1960.0,0.0,-1813.666667,0.0
2020/12/17 19:41:32.199,-1373.0,0.0,-1877.333333,0.0
2020/12/17 19:41:32.307,-845.5,0.0,-1941.000000,0.0


In [87]:
# 1行目は線形補間できないので、「0」とする
df_main.iat[0, 0] = 0
df_main.iat[0, 1] = 0
display(df_main)

,sensor_1,sensor_2,sensor_3,sensor_4
time_stamp,,,,
2020/12/17 19:41:05.098,0.0,0.0,0.000000,-1638.0
2020/12/17 19:41:05.411,-1485.0,0.0,0.000000,-1419.5
2020/12/17 19:41:05.519,-1651.0,0.0,0.000000,-1201.0
2020/12/17 19:41:05.596,-1817.0,0.0,0.000000,-1368.5
2020/12/17 19:41:05.703,-1840.0,0.0,0.000000,-1536.0
...,...,...,...,...
2020/12/17 19:41:31.998,-1960.0,0.0,-1813.666667,0.0
2020/12/17 19:41:32.199,-1373.0,0.0,-1877.333333,0.0
2020/12/17 19:41:32.307,-845.5,0.0,-1941.000000,0.0


In [88]:
# csvファイルに出力
df_main.to_csv("12-8_out.csv")

## Knock119: JSON形式に挑戦

JSON形式のデータ

- 入力データ
  - 12-9.csv
- 加工要件
  - csvデータをJSON形式に変換してファイルに出力する
  - 出力したJSONファイルをデータフレームで読み込む
  - 出力したJSONファイルを辞書型で読み込む
- 出力データ
  - 12-9_out.json
- ヒント
  - 入力データ、要件、出力イメージを確認して必要な処理を考える
  - Pandasでcsvファイルを読み込む
  - JSON形式に変換してファイルとして保存する
  - 保存したJSONファイルをデータフレーム、および辞書型で読み込む

In [89]:
# csvファイルの読み込み
input_data = pd.read_csv("../support/12章/12-9.csv")
display(input_data)

,order_id,customer_id,order_accept_date,total_amount
0,79339111,C26387220,2022-04-01 11:00:00,4144
1,18941733,C48773811,2022-04-01 11:00:00,2877
2,56217880,C24617924,2022-04-01 11:00:00,2603
3,28447783,C26387220,2022-04-01 11:00:00,2732
4,32576156,C54568117,2022-04-01 11:00:00,2987
5,75629806,C38583902,2022-04-30 21:57:57,3050
6,91002809,C48773811,2022-04-30 21:57:57,4692
7,3021273,C24617924,2022-04-30 21:57:57,2388
8,82302078,C26387220,2022-04-30 21:57:57,2603
9,97601615,C54568117,2022-04-30 21:57:57,1899


In [91]:
# JSON形式に変換してファイルを出力する
input_data.to_json("12-9.json")

In [92]:
# JSON形式のファイルを直接データフレームに読み込む
read_json = pd.read_json("12-9.json")
display(read_json)

,order_id,customer_id,order_accept_date,total_amount
0,79339111,C26387220,2022-04-01 11:00:00,4144
1,18941733,C48773811,2022-04-01 11:00:00,2877
2,56217880,C24617924,2022-04-01 11:00:00,2603
3,28447783,C26387220,2022-04-01 11:00:00,2732
4,32576156,C54568117,2022-04-01 11:00:00,2987
5,75629806,C38583902,2022-04-30 21:57:57,3050
6,91002809,C48773811,2022-04-30 21:57:57,4692
7,3021273,C24617924,2022-04-30 21:57:57,2388
8,82302078,C26387220,2022-04-30 21:57:57,2603
9,97601615,C54568117,2022-04-30 21:57:57,1899


In [93]:
# JSON形式のファイルを辞書型で読み込む
with open("12-9.json") as f:
    dict_json = json.load(f)

display(dict_json)

{'order_id': {'0': 79339111,
  '1': 18941733,
  '2': 56217880,
  '3': 28447783,
  '4': 32576156,
  '5': 75629806,
  '6': 91002809,
  '7': 3021273,
  '8': 82302078,
  '9': 97601615,
  '10': 87676506,
  '11': 65713874,
  '12': 80343997,
  '13': 1798443,
  '14': 96208608,
  '15': 5060741,
  '16': 70509759,
  '17': 89047729,
  '18': 25546308,
  '19': 91693481,
  '20': 57908119,
  '21': 7145625,
  '22': 31301192},
 'customer_id': {'0': 'C26387220',
  '1': 'C48773811',
  '2': 'C24617924',
  '3': 'C26387220',
  '4': 'C54568117',
  '5': 'C38583902',
  '6': 'C48773811',
  '7': 'C24617924',
  '8': 'C26387220',
  '9': 'C54568117',
  '10': 'C54568117',
  '11': 'C48773811',
  '12': 'C27698225',
  '13': 'C26387220',
  '14': 'C48773811',
  '15': 'C24617924',
  '16': 'C24617924',
  '17': 'C26387220',
  '18': 'C24617924',
  '19': 'C48773811',
  '20': 'C26387220',
  '21': 'C48773811',
  '22': 'C48773811'},
 'order_accept_date': {'0': '2022-04-01 11:00:00',
  '1': '2022-04-01 11:00:00',
  '2': '2022-04-0

## Knock120: SQLiteに挑戦

SQLite

- 入力データ
  - 12-9.csv
- 加工要件
  - SQLiteを用いてデータベースを構築する
  - csvデータを読み込み、SQLiteデータベースにテーブルを構築する
  - 構築したテーブルにcsvの内容を格納する
  - SQLでデータをデータフレームに格納して表示・保存する
- 出力データ
  - 12-10_out.csv
- ヒント
  - 入力データ、要件、出力イメージを確認して必要な処理を考える
  - SQLiteのライブラリをインポートする
  - Pandasでcsvファイルを読み込む
  - csvのカラムと同じテーブルにデータを格納する
  - データベースからSQLでデータを抽出して出力する

In [95]:
# データベース名（任意のデータベース名.db）
db_name = "TrialDatabase.db"

# データベースに接続
# データベースが存在しない場合、データベースを生成して接続する
con = sqlite3.connect(db_name)
cur = con.cursor()

In [96]:
# csvファイルの読み込み
input_data = pd.read_csv("../support/12章/12-9.csv")
display(input_data)

,order_id,customer_id,order_accept_date,total_amount
0,79339111,C26387220,2022-04-01 11:00:00,4144
1,18941733,C48773811,2022-04-01 11:00:00,2877
2,56217880,C24617924,2022-04-01 11:00:00,2603
3,28447783,C26387220,2022-04-01 11:00:00,2732
4,32576156,C54568117,2022-04-01 11:00:00,2987
5,75629806,C38583902,2022-04-30 21:57:57,3050
6,91002809,C48773811,2022-04-30 21:57:57,4692
7,3021273,C24617924,2022-04-30 21:57:57,2388
8,82302078,C26387220,2022-04-30 21:57:57,2603
9,97601615,C54568117,2022-04-30 21:57:57,1899


In [97]:
# csvの情報をテーブルに格納する
input_data.to_sql("t_data", con,  if_exists="replace", index=None)

23

In [98]:
# 抽出クエリ（SELECT）でデータベースに問い合わせ、
# 結果をデータフレーム型で受け取る
sql = "select * from t_data;"
df = pd.read_sql_query(sql, con)
display(df)

,order_id,customer_id,order_accept_date,total_amount
0,79339111,C26387220,2022-04-01 11:00:00,4144
1,18941733,C48773811,2022-04-01 11:00:00,2877
2,56217880,C24617924,2022-04-01 11:00:00,2603
3,28447783,C26387220,2022-04-01 11:00:00,2732
4,32576156,C54568117,2022-04-01 11:00:00,2987
5,75629806,C38583902,2022-04-30 21:57:57,3050
6,91002809,C48773811,2022-04-30 21:57:57,4692
7,3021273,C24617924,2022-04-30 21:57:57,2388
8,82302078,C26387220,2022-04-30 21:57:57,2603
9,97601615,C54568117,2022-04-30 21:57:57,1899


In [ ]:
# csvファイルに出力する
df.to_csv("12-10_out.csv", index=False)